# [CDAF] Atividade 4

## Nome e matrícula
Nome: Luís Felipe Ramos Ferreira
Matrícula: 2019022553

## Referências
- [1] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.ExpectedThreat.html#socceraction.xthreat.ExpectedThreat
- [3] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.get_successful_move_actions.html#socceraction.xthreat.get_successful_move_actions
- [4] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/xT.html

In [31]:
# Importando bibliotecas
from tqdm import tqdm
import numpy as np
import pandas as pd
import socceraction.spadl as spd
from socceraction import xthreat as xt
import sklearn

### LaLiga  p/ SPADL com pré-processamentos

In [16]:
# carregando os eventos
# neste caso, usaremos a mesma base de dados da atv03
path: str = "../data/atv03/events/events_Spain.json"
events: pd.DataFrame = pd.read_json(path_or_buf=path)

In [17]:
# pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
events = events.rename(columns={'id': 'event_id', 'eventId': 'type_id', 'subEventId': 'subtype_id',
                                'teamId': 'team_id', 'playerId': 'player_id', 'matchId': 'game_id'})
events['milliseconds'] = events['eventSec'] * 1000
events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

In [18]:
# carregando as partidas, pois vamos saber quais times jogam em casa e fora p/ usar como parametro do SPADL
path: str = "../data/atv03/matches/matches_Spain.json"
matches: pd.DataFrame = pd.read_json(path_or_buf=path)

In [19]:
# as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
team_matches: list = []
for i in tqdm(range(len(matches))):
    match = pd.DataFrame(matches.loc[i, "teamsData"]).T
    match['matchId'] = matches.loc[i, "wyId"]
    team_matches.append(match)
team_matches = pd.concat(team_matches).reset_index(drop=True)

100%|██████████| 380/380 [00:00<00:00, 2656.63it/s]


In [20]:
# fazendo a conversão p/ SPADL, padronizando a direção de jogo da esquerda p/ a direita e adicionando os nomes dos tipos de ações
spadl: list = []
game_ids: list = events.game_id.unique().tolist()
for g in tqdm(game_ids):
    match_events = events.loc[events.game_id == g]
    match_home_id = team_matches.loc[(team_matches.matchId == g) & (team_matches.side == 'home'), 'teamId'].values[0]
    match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
    match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
    match_actions = spd.add_names(match_actions)
    spadl.append(match_actions)
spadl = pd.concat(spadl).reset_index(drop=True)

100%|██████████| 380/380 [01:31<00:00,  4.17it/s]


In [21]:
# adicionando o nome dos jogadores
path: str = "../data/atv03/players/players.json"
players: pd.DataFrame = pd.read_json(path_or_buf=path)
players["player_name"] = players["firstName"] + ' ' + players["lastName"]
players = players[["wyId", "player_name"]].rename(columns={"wyId": "player_id"})
spadl = spadl.merge(players, on="player_id", how="left")
spadl

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id,type_name,result_name,bodypart_name,player_name
0,2565548,1,2.994582,682,3542,38.85,26.52,52.50,34.00,180864419,0,0,1,0,pass,success,foot,Manuel Trigueros Mu\u00f1oz
1,2565548,1,3.137020,682,274435,52.50,34.00,47.25,47.60,180864418,0,0,1,1,pass,success,foot,Enes \u00dcnal
2,2565548,1,6.709668,682,364860,47.25,47.60,39.90,59.84,180864420,0,0,1,2,pass,success,foot,Rodrigo Hern\u00e1ndez Cascante
3,2565548,1,8.805497,682,3534,39.90,59.84,33.60,21.08,180864421,0,0,1,3,pass,success,foot,Jaume Vicent Costa Jord\u00e1
4,2565548,1,14.047492,682,3695,33.60,21.08,32.55,42.84,180864422,0,0,1,4,pass,success,foot,\u00c1lvaro Gonz\u00e1lez Sober\u00f3n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473889,2565927,2,2939.077491,682,20623,69.30,51.00,92.40,66.64,253302671,0,0,1,1482,pass,success,foot,Roberto Soriano
473890,2565927,2,2940.515560,682,122832,92.40,66.64,101.85,53.72,253302673,0,21,1,1483,dribble,success,foot,Salem Mohammed Al Dawsari
473891,2565927,2,2942.098761,682,122832,101.85,53.72,96.60,50.32,253302674,0,1,0,1484,cross,fail,foot,Salem Mohammed Al Dawsari
473892,2565927,2,2943.089232,675,40756,8.40,17.68,9.45,29.92,253302698,0,0,1,1485,pass,success,foot,Carlos Henrique Casimiro


## Questão 1
- Crei um dataframe "shots" à partir do dataframe "spadl", contendo apenas os chutes.
- Crie 4 colunas no dataframe "shots" a serem usadas como features de um modelo de xG.
- Justifique a escolha das features.

In [50]:
GOAL_CENTER_X: int = 105
GOAL_CENTER_Y: int = 34

UPPER_CROSSBAR_X: int = 105
UPPER_CROSSBAR_Y: int = 38

LOWER_CROSSBAR_X: int = 105
LOWER_CROSSBAR_Y: int = 30

In [55]:
shots: pd.DataFrame = spadl.query('type_name == "shot"').copy()

shots["distance_to_goal"] = np.sqrt((shots["start_x"] - GOAL_CENTER_X) ** 2 - (shots["start_y"] - GOAL_CENTER_Y))

vector_1 = [UPPER_CROSSBAR_X - shots["start_x"], UPPER_CROSSBAR_Y - shots["start_y"]]
vector_2: np.array = [LOWER_CROSSBAR_X - shots["start_x"], LOWER_CROSSBAR_Y - shots["start_y"]]
shots["angle_of_shot"] = np.arccos(np.dot(vector_1 / np.linalg.norm(vector_1), vector_2 / np.linalg.norm(vector_2)))

shots["number_of_opponents_in_front"] = np.nan
shots["shot_capacity"] = np.nan
shots


/home/lfrf/anaconda3/envs/futebol/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


list

- dis to goal: consideramos ponto do gol sendo 105, 34
- angle of shot: angulo entre bola e traves, sendo estes respectivamente 105, 30 e 105, 38
- 

## Questão 2
- Crie uma coluna numérica binária "goal" no dataframe "shots" indicando se o chute resultou em gol ou não.
- Use regressão logística [1] p/ treinar (.fit(X_train, y_train)) um modelo de xG usando as features criadas na questão 1.
- Use 70% dos dados para treino e 30% para teste.
- Reporte a acurácia do modelo para os conjuntos de treino (.score(X_train, y_train)) e teste (.score(X_test, y_test)).

In [32]:
shots["goal"] = shots["result_name"].apply(lambda res: int(res == "success"))
shots

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,...,action_id,type_name,result_name,bodypart_name,player_name,distance_to_goal,angle_of_shot,number_of_opponents_in_front,shot_capacity,goal
20,2565548,1,57.771186,695,225089,97.65,44.88,105.00,34.00,180865315,...,20,shot,fail,foot,Jos\u00e9 Luis Morales Nogales,NaN,NaN,NaN,NaN,0
22,2565548,1,60.727239,695,255738,84.00,27.88,84.00,27.88,180864547,...,22,shot,fail,foot,Jefferson Andr\u00e9s Lerma Sol\u00eds,NaN,NaN,NaN,NaN,0
93,2565548,1,446.986112,682,37831,92.40,29.24,92.40,29.24,180864486,...,93,shot,fail,foot,Carlos Arturo Bacca Ahumada,NaN,NaN,NaN,NaN,0
96,2565548,1,488.929113,682,15214,91.35,23.12,105.00,27.20,180864491,...,96,shot,fail,foot,Antonio Rukavina,NaN,NaN,NaN,NaN,0
178,2565548,1,948.872079,695,225089,78.75,40.80,105.00,34.00,180864792,...,178,shot,fail,foot,Jos\u00e9 Luis Morales Nogales,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473633,2565927,2,1776.027412,682,267134,97.65,23.12,97.65,23.12,253302272,...,1226,shot,fail,foot,Roger Beyker Mart\u00ednez Tobinson,NaN,NaN,NaN,NaN,0
473673,2565927,2,1944.188119,682,267134,94.50,46.24,94.50,46.24,253302329,...,1266,shot,fail,foot,Roger Beyker Mart\u00ednez Tobinson,NaN,NaN,NaN,NaN,0
473806,2565927,2,2385.837008,682,134174,96.60,51.00,105.00,37.40,253302547,...,1399,shot,success,foot,Samuel Castillejo Azuaga,NaN,NaN,NaN,NaN,1
473851,2565927,2,2722.835144,675,3321,86.10,47.60,105.00,30.60,253302642,...,1444,shot,fail,foot,Karim Benzema,NaN,NaN,NaN,NaN,0


## Questão 3
- Use o modelo treinado na questão 2 p/ prever a probabilidade de gol de todos os chutes do dataframe "shots". Reporte essas probabilidades no dataframe "shots" em uma coluna "xG".
- Agrupe o dataframe "shots" por "player_name" e reporte a soma dos "goal" e "xG".
- Reporte os 10 jogadores com maior xG.
- Reporte os 10 jogadores com maior diferença de Gols e xG.

## Questão 4 [4]
- Instancie um objeto ExpectedThreat [2] com parâmetros l=25 e w=16.
- Faça o fit do modelo ExpectedThreat com o dataframe "spadl".

## Questão 5
- Crie um dataframe "prog_actions" à partir do dataframe "spadl", contendo apenas as ações de progressão e que são bem-sucedidas [3].
- Use o método rate do objeto ExpectedThreat p/ calcular o valor de cada ação de progressão do dataframe "prog_actions", em uma coluna chamada "action_value".
- Agrupe o dataframe "prog_actions" por "player_name" e reporte a soma dos "action_value".
- Reporte os 10 jogadores com maior "action_value".